In [1]:
import os
import numpy as np

from flow.controllers.base_controller import BaseController
from flow.scenarios.merge import MergeScenario, ADDITIONAL_NET_PARAMS
from flow.core.params import VehicleParams, NetParams, InitialConfig, TrafficLightParams, InFlows, SumoParams, EnvParams, SumoCarFollowingParams
from flow.envs.merge import WaveAttenuationMergePOEnv, ADDITIONAL_ENV_PARAMS as WA_ADDITIONAL_ENV_PARAMS
from flow.envs.loop.loop_accel import AccelEnv, ADDITIONAL_ENV_PARAMS as LA_ADDITIONAL_ENV_PARAMS
from flow.core.experiment import Experiment

In [2]:
class IDMController(BaseController):
    def __init__(self, veh_id, v0=30, T=1, a=1, b=1.5, 
                 delta=4, s0=2, s1=0, time_delay=0.0, 
                 dt=0.1, noise=0, fail_safe=None, car_following_params=None):
        """
        veh_id: str
            unique vehicle identifier
        car_following_params: SumoCarFollowingParams
            see parent class
        v0: float, optional
            desirable velocity, in m/s (default: 30)
        T: float, optional
            safe time headway, in s (default: 1)
        b: float, optional
            comfortable deceleration, in m/s2 (default: 1.5)
        delta: float, optional
            acceleration exponent (default: 4)
        s0: float, optional
            linear jam distance, in m (default: 2)
        s1: float, optional
            nonlinear jam distance, in m (default: 0)
        dt: float, optional
            timestep, in s (default: 0.1)
        noise: float, optional
            std dev of normal perturbation to the acceleration (default: 0)
        fail_safe: str, optional
            type of flow-imposed failsafe the vehicle should posses, defaults
            to no failsafe (None)
        """
        BaseController.__init__(self, veh_id, car_following_params,
                                delay=time_delay, fail_safe=fail_safe,
                                noise=noise)
        self.v0 = v0
        self.T = T
        self.a = a
        self.b = b
        self.delta = delta
        self.s0 = s0
        self.s1 = s1
        self.dt = dt

    def get_accel(self, env):
        v = env.k.vehicle.get_speed(self.veh_id)
        lead_id = env.k.vehicle.get_leader(self.veh_id)
        h = env.k.vehicle.get_headway(self.veh_id)

        # negative headways may be registered by sumo at intersections/
        # junctions. Setting them to 0 causes vehicles to not move; therefore,
        # we maintain these negative headways to let sumo control the dynamics
        # as it sees fit at these points.
        if abs(h) < 1e-3:
            h = 1e-3

        if lead_id is None or lead_id == '':  # no car ahead
            s_star = 0
        else:
            lead_vel = env.k.vehicle.get_speed(lead_id)
            s_star = self.s0 + max(
                0,
                v * self.T + v*(v-lead_vel) / (2*np.sqrt(self.a*self.b)))

        return self.a * (1 - (v/self.v0)**self.delta - (s_star/h)**2)

## Add Vehicles to the Scenario
These vehicle will be controlled by whatever acceleration controller set to.

In [3]:
name = "merge_example"
vehicles = VehicleParams()
vehicles.add("human",
             acceleration_controller=(IDMController, {}),
             car_following_params=SumoCarFollowingParams(
                 speed_mode="obey_safe_speed"))

## Add Inflows to the Scenario

In [4]:
inflow = InFlows()

# Inflow for highway lanes
inflow.add(veh_type="human",
            edge="inflow_highway",
            vehs_per_hour=10000,
            depart_lane="random",
            depart_speed=ADDITIONAL_NET_PARAMS['speed_limit'],
            color="white")

# Inflow for right merge lane
inflow.add(veh_type="human",
            edge="inflow_merge",
            period=2,
            depart_lane=0,  # right lane
            depart_speed=0,
            color="green")

# Inflow for left merge lane
inflow.add(veh_type="human",
           edge="inflow_merge",
           probability=0.1,
           depart_lane=1,  # left lane
           depart_speed="random",
           begin=60,  # 1 minute
           number=30,
           color="red")

## Set the Net Params
These control the configuration of the highway merge network.

In [5]:
# Optionally adjust the net params
additional_net_params = ADDITIONAL_NET_PARAMS.copy()
additional_net_params['merge_length'] = 100 # length of the merge edge
additional_net_params['pre_merge_length'] = 200 # length of the highway leading to the merge
additional_net_params['post_merge_length'] = 350  # length of the highway past the merge
additional_net_params['merge_lanes'] = 2 # number of lanes in the merge
additional_net_params['highway_lanes'] = 4 # number of lanes in the highway
additional_net_params['speed_limit'] = 30 # max speed limit of the network

net_params = NetParams(inflows=inflow,
                       additional_params=additional_net_params)

## Set the Initial Configuration Params
These parameters that affect the positioning of vehicle in the network at the start of a rollout. By default, vehicles are uniformly distributed in the network.

In [6]:
initial_config = InitialConfig(shuffle=False,
                               spacing="uniform",
                               min_gap=0,
                               perturbation=1,
                               x0=0,
                               bunching=0,
                               lanes_distribution=float("inf"),
                               edges_distribution="all",
                               additional_params=None)

## Set the Traffic Light Params
This class is used to place traffic lights in the network and describe the state of these traffic lights. In addition, this class supports modifying the states of certain lights via TraCI.

In [7]:
traffic_lights = TrafficLightParams()

## Set the Simulation Params
These include the simulation time step, whether or not to render the simulation, and where to write the output results.

In [8]:
sumo_params = SumoParams(sim_step=0.1, render=True, emission_path='data')

## Set the Environment Params
We can use the WaveAttenuationEnv or the AccelEnv.

In [9]:
# Optionally adjust the env params
USE_WA_ENV = True

additional_env_params = None
if USE_WA_ENV:
    additional_env_params = WA_ADDITIONAL_ENV_PARAMS.copy()
    additional_env_params['max_accel'] = 3
    additional_env_params['max_decel'] = 3
    additional_env_params['target_velocity'] = 25
    additional_env_params['num_rl'] = 5
else:
    additional_env_params = LA_ADDITIONAL_ENV_PARAMS.copy()
    additional_env_params['max_accel'] = 3
    additional_env_params['max_decel'] = 3
    additional_env_params['target_velocity'] = 25
    additional_env_params['sort_vehicles'] = False

env_params = EnvParams(additional_params=additional_env_params,
                       horizon=float('inf'),
                       warmup_steps=0,
                       sims_per_step=1,
                       evaluate=False,
                       clip_actions=True)

## Run the Experiment

In [10]:
# create the scenario object
scenario = MergeScenario(name="merge_example",
                         vehicles=vehicles,
                         net_params=net_params,
                         initial_config=initial_config,
                         traffic_lights=traffic_lights)

# create the environment object
env = None
if USE_WA_ENV:
    env = WaveAttenuationMergePOEnv(env_params, sumo_params, scenario)
else:
    env = AccelEnv(env_params, sumo_params, scenario)

# create the experiment object
exp = Experiment(env)

# run the experiment for a set number of rollouts / time steps
_ = exp.run(1, 3000, convert_to_csv=True)

**********************************************************
**********************************************************
**********************************************************
significantly decrease after large number of rollouts. In 
order to avoid this, set SumoParams(restart_instance=True).
**********************************************************
**********************************************************
**********************************************************


/home/jacob/anaconda3/envs/flow/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/jacob/anaconda3/envs/flow/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Round 0, return: 1276.2515333291042
Average, std return: 1276.2515333291042, 0.0
Average, std speed: nan, nan


In [11]:
emission_location = os.path.join(exp.env.sim_params.emission_path, exp.env.scenario.name)
print(emission_location + '-emission.csv')

data/merge_example_20220302-1202461646240566.8454936-emission.csv
